#  Project Title: Optimizing Server Procurement for Phased Intranet Deployment


##  Problem Overview

A logistic company is facing a significant internal communication breakdowns, leading to missed orders, customer dissatisfaction, and declining stock value. To address this, the company decided to implement a phased intranet rollout over five months—onboarding departments one after another with a budget constraints.


However, purchasing servers to support new users each month is constrained by:

- Budget caps in Months 1–2 ($9,500 max),

- Available discounts on SGI (10%) and Sun (25%) servers in Months 1–2,

- Varying server capacities and costs,

- A critical constraint that Month 3 (Manufacturing) **must** receive at least one high-performance server (Enhanced, SGI, or Sun).


## 🎯 Objective

To **minimize the total cost of server procurement** while meeting monthly user demand and all operational constraints through:

- **Month-by-month optimization** where each month is treated independently,

- **Cumulative planning** allowing servers bought early to serve future demand.


### Month-by-month optimization where each month is treated independently.

In [54]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

# Define server cost and capacity data (using the table values)
cost = {
    "Standard": 2500,
    "Enhanced": 5000,
    "SGI": 10000,
    "Sun": 25000
}

capacity = {
    "Standard": 30,
    "Enhanced": 80,
    "SGI": 200,
    "Sun": 2000
}

# Define new employees for each month 
#Month 1 is omitted because it is 0

# Month 2: Sales (50), Month 3: Manufacturing (180), Month 4: Warehouse (30), Month 5: Marketing (70)
month_details = {
    2: {"employees": 50, "constraints": ["Budget"]},       # Month 2 includes a budget constraint (M1+M2 available funds)
    3: {"employees": 180, "constraints": ["Manufacturing"]}, # Manufacturing requirement applies in Month 3
    4: {"employees": 30, "constraints": []},
    5: {"employees": 70, "constraints": []}
}

# Dictionary to store each month's solution
solutions = {}
total_cost = 0

# Loop through the months and create a separate model for each
for m in sorted(month_details.keys()):
    emp_req = month_details[m]["employees"]
    cons = month_details[m]["constraints"]
    
    # Create a new model for this month
    model_name = f"Month_{m}_Purchase"
    prob = LpProblem(model_name, LpMinimize)
    
    # Define decision variables with descriptive names
    Standard_Intel = LpVariable("Standard_Intel", lowBound=0, cat='Integer')
    Enhanced_Intel = LpVariable("Enhanced_Intel", lowBound=0, cat='Integer')
    SGI_Workstation = LpVariable("SGI_Workstation", lowBound=0, cat='Integer')
    Sun_Workstation = LpVariable("Sun_Workstation", lowBound=0, cat='Integer')
    
    # Objective function: minimize cost
    prob += ( cost["Enhanced"] * Enhanced_Intel +
              cost["SGI"] * SGI_Workstation +
              cost["Standard"] * Standard_Intel +
              cost["Sun"] * Sun_Workstation ), "Total_Cost"
    
    # Capacity constraint: the total capacity purchased in that month must cover the new users.
    prob += ( capacity["Enhanced"] * Enhanced_Intel +
              capacity["SGI"] * SGI_Workstation +
              capacity["Standard"] * Standard_Intel +
              capacity["Sun"] * Sun_Workstation  >= emp_req ), "Capacity"
    
    # Additional constraints per month:
    if "Budget" in cons:
        prob += ( cost["Enhanced"] * Enhanced_Intel +
                  cost["SGI"] * SGI_Workstation +
                  cost["Standard"] * Standard_Intel +
                  cost["Sun"] * Sun_Workstation <= 9500 ), "Budget_Constraint"
    if "Manufacturing" in cons:
        prob += ( Enhanced_Intel + SGI_Workstation + Sun_Workstation >= 1 ), "Manufacturing_Requirement"
    
    # Solve the model for this month
    prob.solve()
    
    # Save the solution results
    sol = {
        "Standard_Intel": Standard_Intel.varValue,
        "Enhanced_Intel": Enhanced_Intel.varValue,
        "SGI_Workstation": SGI_Workstation.varValue,
        "Sun_Workstation": Sun_Workstation.varValue,
        "Cost": value(prob.objective)
    }
    solutions[m] = sol
    total_cost += sol["Cost"]
    
    # Print the model details (formatted similar to the sample output)
    print(f"\n{model_name}:")
    print("MINIMIZE")
    print(f"  {cost['Enhanced']}*Enhanced_Intel + {cost['SGI']}*SGI_Workstation + {cost['Standard']}*Standard_Intel + {cost['Sun']}*Sun_Workstation + 0")
    print("SUBJECT TO")
    print(f"  Capacity: {capacity['Enhanced']}*Enhanced_Intel + {capacity['SGI']}*SGI_Workstation + {capacity['Standard']}*Standard_Intel + {capacity['Sun']}*Sun_Workstation >= {emp_req}")
    if "Budget" in cons:
        print(f"  Budget_Constraint: {cost['Enhanced']}*Enhanced_Intel + {cost['SGI']}*SGI_Workstation + {cost['Standard']}*Standard_Intel + {cost['Sun']}*Sun_Workstation <= 9500")
    if "Manufacturing" in cons:
        print("  Manufacturing_Requirement: Enhanced_Intel + SGI_Workstation + Sun_Workstation >= 1")
    print("VARIABLES")
    print("  0 <= Enhanced_Intel Integer")
    print("  0 <= SGI_Workstation Integer")
    print("  0 <= Standard_Intel Integer")
    print("  0 <= Sun_Workstation Integer")
    
    # Print the solution for the current month
    print(f"\nSolution for {model_name}:")
    if sol["Enhanced_Intel"] != 0:
        print(f"  Enhanced_Intel - {int(sol['Enhanced_Intel'])}")
    if sol["SGI_Workstation"] != 0:
        print(f"  SGI_Workstation - {int(sol['SGI_Workstation'])}")
    if sol["Standard_Intel"] != 0:
        print(f"  Standard_Intel - {int(sol['Standard_Intel'])}")
    if sol["Sun_Workstation"] != 0:
        print(f"  Sun_Workstation - {int(sol['Sun_Workstation'])}")
    print(f"Cost for {model_name}: ${sol['Cost']}")
    
# Print summary of monthly purchases and total cost
print("\nMonthly Purchases:")
for m in sorted(solutions.keys()):
    print(f"Month {m}:")
    sol = solutions[m]
    if sol["Enhanced_Intel"]:
        print(f"  Enhanced_Intel - {int(sol['Enhanced_Intel'])}")
    if sol["SGI_Workstation"]:
        print(f"  SGI_Workstation - {int(sol['SGI_Workstation'])}")
    if sol["Standard_Intel"]:
        print(f"  Standard_Intel - {int(sol['Standard_Intel'])}")
    if sol["Sun_Workstation"]:
        print(f"  Sun_Workstation - {int(sol['Sun_Workstation'])}")
print(f"Total Cost: ${total_cost}")



Month_2_Purchase:
MINIMIZE
  5000*Enhanced_Intel + 10000*SGI_Workstation + 2500*Standard_Intel + 25000*Sun_Workstation + 0
SUBJECT TO
  Capacity: 80*Enhanced_Intel + 200*SGI_Workstation + 30*Standard_Intel + 2000*Sun_Workstation >= 50
  Budget_Constraint: 5000*Enhanced_Intel + 10000*SGI_Workstation + 2500*Standard_Intel + 25000*Sun_Workstation <= 9500
VARIABLES
  0 <= Enhanced_Intel Integer
  0 <= SGI_Workstation Integer
  0 <= Standard_Intel Integer
  0 <= Sun_Workstation Integer

Solution for Month_2_Purchase:
  Standard_Intel - 2
Cost for Month_2_Purchase: $5000.0

Month_3_Purchase:
MINIMIZE
  5000*Enhanced_Intel + 10000*SGI_Workstation + 2500*Standard_Intel + 25000*Sun_Workstation + 0
SUBJECT TO
  Capacity: 80*Enhanced_Intel + 200*SGI_Workstation + 30*Standard_Intel + 2000*Sun_Workstation >= 180
  Manufacturing_Requirement: Enhanced_Intel + SGI_Workstation + Sun_Workstation >= 1
VARIABLES
  0 <= Enhanced_Intel Integer
  0 <= SGI_Workstation Integer
  0 <= Standard_Intel Integer
  

## - **Cumulative planning** allowing servers bought early to serve future demand.


In [47]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

# Define the model
prob = LpProblem("Cumulative_CommuniCorp_Server_Optimization", LpMinimize)

# There are 5 months (we index them 0 to 4 corresponding to Month 1 to Month 5)
months = range(5)

# Define server types and their data:
server_types = ["Standard", "Enhanced", "SGI", "Sun"]

# Capacity each server provides
capacity = {
    "Standard": 30,
    "Enhanced": 80,
    "SGI": 200,
    "Sun": 2000
}

# Prices: note that discounted prices apply in months 1 and 2 (indices 0 and 1)
cost_discount = {
    "Standard": 2500,          # no discount
    "Enhanced": 5000,          # no discount
    "SGI": 10000 * 0.9,        # 10% off => 9000
    "Sun": 25000 * 0.75        # 25% off => 18750
}
cost_full = {
    "Standard": 2500,
    "Enhanced": 5000,
    "SGI": 10000,
    "Sun": 25000
}

# Cumulative employees required by the end of each month:
# Month 1: 0, Month 2: 50, Month 3: 230, Month 4: 260, Month 5: 330
cum_employees = [0, 50, 230, 260, 330]

# decision variables for each server type and each month.
# For example, Standard_M1 is the number of Standard servers purchased in Month 1.
vars_dict = { st: [LpVariable(f"{st}_M{m+1}", lowBound=0, cat='Integer') for m in months] 
              for st in server_types }

# Constraint: Budget for Months 1 and 2.
# Spending in months 1 and 2 must not exceed $9,500.
budget_expr = lpSum(vars_dict[st][m] * cost_discount[st] 
                    for st in server_types for m in [0, 1])
prob += (budget_expr <= 9500), "Budget_M1_M2"

# Cumulative Capacity Constraints
# For each month m, the total capacity from servers purchased from Month 1 up to m 
# must be at least the cumulative number of employees.
for m in months:
    capacity_expr = lpSum(vars_dict[st][k] * capacity[st] for st in server_types for k in range(m+1))
    prob += (capacity_expr >= cum_employees[m]), f"Capacity_M{m+1}"

# Manufacturing Requirement
# By Month 3 (index 2), the cumulative number of powerful servers (Enhanced, SGI, Sun)
# purchased in months 1–3 must be at least 1.
manufacturing_expr = lpSum(vars_dict[st][k] for st in ["Enhanced", "SGI", "Sun"] for k in range(3))
prob += (manufacturing_expr >= 1), "Manufacturing_Requirement"

# Objective Function
# Minimize the total cost of purchasing servers over all 5 months.
total_cost = 0
for m in months:
    for st in server_types:
        if m in [0, 1]:
            cost_m = cost_discount[st]
        else:
            cost_m = cost_full[st]
        total_cost += vars_dict[st][m] * cost_m
prob += total_cost, "Total_Cost"

# Solve the model
prob.solve()
print(prob)

# Output the results
print("Status:", LpStatus[prob.status])
for st in server_types:
    for m in months:
        print(f"{st}_M{m+1} =", vars_dict[st][m].varValue)
print("Total Cost:", value(prob.objective))


Cumulative_CommuniCorp_Server_Optimization:
MINIMIZE
5000*Enhanced_M1 + 5000*Enhanced_M2 + 5000*Enhanced_M3 + 5000*Enhanced_M4 + 5000*Enhanced_M5 + 9000.0*SGI_M1 + 9000.0*SGI_M2 + 10000*SGI_M3 + 10000*SGI_M4 + 10000*SGI_M5 + 2500*Standard_M1 + 2500*Standard_M2 + 2500*Standard_M3 + 2500*Standard_M4 + 2500*Standard_M5 + 18750.0*Sun_M1 + 18750.0*Sun_M2 + 25000*Sun_M3 + 25000*Sun_M4 + 25000*Sun_M5 + 0.0
SUBJECT TO
Budget_M1_M2: 5000 Enhanced_M1 + 5000 Enhanced_M2 + 9000 SGI_M1 + 9000 SGI_M2
 + 2500 Standard_M1 + 2500 Standard_M2 + 18750 Sun_M1 + 18750 Sun_M2 <= 9500

Capacity_M1: 80 Enhanced_M1 + 200 SGI_M1 + 30 Standard_M1 + 2000 Sun_M1 >= 0

Capacity_M2: 80 Enhanced_M1 + 80 Enhanced_M2 + 200 SGI_M1 + 200 SGI_M2
 + 30 Standard_M1 + 30 Standard_M2 + 2000 Sun_M1 + 2000 Sun_M2 >= 50

Capacity_M3: 80 Enhanced_M1 + 80 Enhanced_M2 + 80 Enhanced_M3 + 200 SGI_M1
 + 200 SGI_M2 + 200 SGI_M3 + 30 Standard_M1 + 30 Standard_M2 + 30 Standard_M3
 + 2000 Sun_M1 + 2000 Sun_M2 + 2000 Sun_M3 >= 230

Capacit

## ✅ Results & Conclusion


### ✔️ Monthly Model

- **Month 2 (Sales):** 2 Standard servers – $5,000  

- **Month 3 (Manufacturing):** 1 SGI server – $10,000  

- **Month 4 (Warehouse):** 1 Standard server – $2,500  

- **Month 5 (Marketing):** 1 Enhanced server – $5,000  

- **🟡 Total Cost:** **$22,500**


> 💡 *Each month is treated separately, leading to underutilization of capacity and missed discount opportunities.*


### ✔️ Cumulative Model 

- **Month 1:** 1 SGI server (discounted) – $9,000  

- **Month 3:** 1 SGI server (full price) – $10,000  

- **Months 2, 4, 5:** No additional purchase (existing capacity suffices)  

- **🟢 Total Cost:** **$19,000**


> 🏆 *A more cost-effective solution, leveraging early purchases and discounts to satisfy later demand.*


## 💡 Conclusion


### The **cumulative approach** proved significantly more efficient, saving **$3,500** over the month-by-month model by exploiting early discounts and sharing capacity across time. This case illustrates the power of holistic planning and optimization techniques like **integer programming** in solving real-world operational challenges. It also emphasizes the importance of **interdepartmental coordination**, timely investment, and strategic resource allocation.
